In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
#device = torch.device("cuda" if use_gpu else "cpu")  
    
    
batch_size = 128
model_name = "Resnet_quant"
model = resnet20_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()

In [ ]:
# This cell won't be given, but students will complete the training

lr = 4.4e-2
weight_decay = 1e-4
epochs = 180
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [2]:
PATH = "result/Resnet_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9178/10000 (92%)



In [3]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self,module,module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []
        
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, nn.Conv2d):
        layer.register_forward_pre_hook(save_output)

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)


In [4]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha 
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q / w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-0.0000,  1.0000,  3.0000],
          [-2.0000, -1.0000,  2.0000],
          [-1.0000,  0.0000,  2.0000]],

         [[-1.0000, -1.0000, -1.0000],
          [ 2.0000, -2.0000, -7.0000],
          [ 3.0000,  3.0000,  1.0000]],

         [[ 5.0000,  4.0000, -3.0000],
          [ 7.0000,  7.0000,  3.0000],
          [-0.0000,  1.0000, -2.0000]],

         ...,

         [[-1.0000, -3.0000, -4.0000],
          [-2.0000, -4.0000, -3.0000],
          [-2.0000, -2.0000,  0.0000]],

         [[-4.0000,  2.0000,  2.0000],
          [-4.0000,  4.0000,  7.0000],
          [ 0.0000,  4.0000, -0.0000]],

         [[ 2.0000,  1.0000, -0.0000],
          [ 2.0000, -0.0000, -2.0000],
          [ 2.0000, -0.0000, -2.0000]]],


        [[[ 2.0000,  4.0000,  6.0000],
          [-1.0000,  3.0000,  4.0000],
          [-4.0000, -0.0000, -0.0000]],

         [[ 2.0000, -1.0000,  1.0000],
          [-3.0000,  1.0000, -2.0000],
          [ 4.0000,  6.0000,  3.0000]],

         [[ 1.0000, -0.0000,  1

In [5]:
print(model)
x_bit = 4
x = save_output.outputs[1][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv1.act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2

In [6]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int * w_delta * x_delta
print(output_recovered)

tensor([[[[-4.5160e+00, -2.3440e+01, -3.3118e+01,  ..., -3.6559e+00,
            5.0178e-01,  3.5125e+00],
          [ 1.2903e+00, -1.9785e+01, -2.6236e+01,  ..., -4.8745e+00,
            4.5161e+00,  6.2365e+00],
          [ 1.5054e+00, -1.9570e+01, -2.5806e+01,  ..., -2.0071e+00,
            5.9497e+00,  9.1038e+00],
          ...,
          [ 4.3010e-01, -2.7885e+01, -4.3512e+01,  ...,  1.2831e+01,
            6.5949e+00,  9.6056e+00],
          [-1.1469e+00, -3.0752e+01, -3.0967e+01,  ...,  2.7240e+00,
            1.3620e+00,  4.8028e+00],
          [-3.3691e+00, -2.4444e+01, -1.7204e+01,  ..., -5.0178e+00,
           -4.7311e+00, -7.4551e+00]],

         [[ 1.1254e+01,  1.9211e+01,  1.0394e+01,  ...,  6.2365e+00,
            3.0107e+00, -3.5125e+00],
          [ 1.7634e+01,  2.3297e+01,  1.8208e+01,  ...,  6.4515e-01,
            2.2939e+00, -3.5842e-01],
          [ 1.7993e+01,  2.3799e+01,  1.8494e+01,  ..., -3.4408e+00,
            1.0322e+01,  1.3476e+01],
          ...,
     

In [7]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q 

output_ref = conv_ref(x)
print(output_ref)

tensor([[[[-4.6597e+00, -2.5733e+01, -3.2835e+01,  ..., -3.5135e+00,
            1.9584e+00,  3.5007e+00],
          [ 7.9587e-01, -2.1969e+01, -2.5796e+01,  ..., -5.0442e+00,
            6.1554e+00,  6.2092e+00],
          [ 1.1467e+00, -2.0376e+01, -2.5855e+01,  ..., -1.1504e+00,
            5.8581e+00,  9.1889e+00],
          ...,
          [-2.9927e+00, -3.3437e+01, -4.7374e+01,  ...,  1.8183e+01,
            1.3683e+01,  1.1037e+01],
          [-4.9380e+00, -3.9532e+01, -3.7526e+01,  ...,  3.8721e+00,
            2.2386e+00,  1.7452e+00],
          [-4.9245e+00, -2.8319e+01, -2.0530e+01,  ..., -8.0153e+00,
           -7.3359e+00, -1.1229e+01]],

         [[ 1.1465e+01,  1.9983e+01,  1.2527e+01,  ...,  5.0320e+00,
            4.1526e+00, -4.0398e+00],
          [ 1.7109e+01,  2.3046e+01,  2.0405e+01,  ..., -4.4969e-01,
            8.2300e-01, -6.0840e-01],
          [ 1.7087e+01,  2.3309e+01,  1.9142e+01,  ..., -3.5624e+00,
            1.0037e+01,  1.3501e+01],
          ...,
     

In [8]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-2.2607e+00, -2.4806e+01, -3.1008e+01,  ..., -3.5973e+00,
            1.8554e+00,  3.5235e+00],
          [ 3.3864e+00, -2.0572e+01, -2.4636e+01,  ..., -4.9534e+00,
            6.4759e+00,  6.3380e+00],
          [ 3.7171e+00, -1.9050e+01, -2.4396e+01,  ..., -7.8262e-01,
            6.0045e+00,  8.5060e+00],
          ...,
          [-3.5722e-01, -3.1624e+01, -4.4902e+01,  ...,  1.8069e+01,
            1.3894e+01,  1.1173e+01],
          [-2.1447e+00, -3.7620e+01, -3.6145e+01,  ...,  4.2146e+00,
            2.4184e+00,  2.2785e+00],
          [-2.7171e+00, -2.7564e+01, -1.9787e+01,  ..., -5.8661e+00,
           -5.2428e+00, -8.9488e+00]],

         [[ 1.0944e+01,  2.2099e+01,  1.1629e+01,  ...,  4.0247e+00,
            2.8597e+00, -5.2097e+00],
          [ 1.7111e+01,  2.5431e+01,  2.0005e+01,  ..., -1.0258e+00,
           -1.6340e-01, -1.6222e+00],
          [ 1.7063e+01,  2.5619e+01,  1.8543e+01,  ..., -3.5446e+00,
            9.2673e+00,  1.2805e+01],
          ...,
     

In [9]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.4554, device='cuda:0', grad_fn=<MeanBackward0>)
